# Install the [hotsline](https://github.com/AntoineGrimaldi/hotsline) package

In [1]:
!python3 --version

Python 3.8.10


In [2]:
%cd ~/Documents/projets/HOTS/
!git clone git@github.com:AntoineGrimaldi/hotsline.git
%cd hotsline/
!git pull
!python3 -m pip install --user -r requirements.txt
!python3 -m pip install --user -e .
%cd ../aprovhots/tutorials/

/home/INT/grimaldi.a/Documents/projets/HOTS
fatal: le chemin de destination 'hotsline' existe déjà et n'est pas un répertoire vide.
/home/INT/grimaldi.a/Documents/projets/HOTS/hotsline
Déjà à jour.
Obtaining file:///home/INT/grimaldi.a/Documents/projets/HOTS/hotsline
  Preparing metadata (setup.py) ... done
  Attempting uninstall: hots
    Found existing installation: hots 1.0
    Uninstalling hots-1.0:
      Successfully uninstalled hots-1.0
  Running setup.py develop for hots
/home/INT/grimaldi.a/Documents/projets/HOTS/aprovhots/tutorials


<span style="color:red">**<<<<<<< local**</span>

In [3]:
import tonic, os
from hots.utils import get_loader
from hots.network import network

# Make a trial of the package with Poker DVS

In [4]:
transform = tonic.transforms.NumpyAsType(int)
trainset = tonic.datasets.POKERDVS(save_to='../../Data/', train=True, transform=transform)
testset = tonic.datasets.POKERDVS(save_to='../../Data/', train=False, transform=transform)
trainloader = get_loader(trainset)
testloader = get_loader(testset)
num_sample_train = len(trainloader)
num_sample_test = len(testloader)
print(f'number of samples in the training set: {len(trainloader)}')
print(f'number of samples in the testing set: {len(testloader)}')

number of samples in the training set: 48
number of samples in the testing set: 20


## Initialization of the network

In [5]:
name = 'homeohots'
homeo = True
timestr = '2022-05-18'
dataset_name = 'poker'

Rz = [2, 4]
N_neuronz = [8, 16]
tauz = [1e3, 4e3]

hots = network(name, dataset_name, timestr, trainset.sensor_size, nb_neurons = N_neuronz, tau = tauz, R = Rz, homeo = homeo)

device -> cuda


## Unsupervised clustering

In [6]:
if not os.path.exists('../Records/'):
    os.mkdir('../Records/')
    os.mkdir('../Records/networks/')
    os.mkdir('../Records/LR_results/')
path = '../Records/networks/'+hots.name+'.pkl'
if not os.path.exists(path):
    hots.clustering(trainloader, trainset.ordering, record = True)
hots.plotlayers();
hots.plotlearning();

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [05:22<00:00,  6.72s/it]


FileNotFoundError: [Errno 2] No such file or directory: '../Records/networks/2022-05-18_poker_homeohots_True_[8, 16]_[1000.0, 4000.0]_[2, 4].pkl'

## Coding of the training and testing set

In [ ]:
hots.coding(trainloader, trainset.ordering, trainset.classes, training=True)
hots.coding(testloader, trainset.ordering, trainset.classes, training=False)

this dataset have already been processed, check at: 
 ../Records/output/train/2022-05-18_poker_homeohots_True_[8, 16]_[1000.0, 4000.0]_[2, 4]_48_(None, None)/
this dataset have already been processed, check at: 
 ../Records/output/test/2022-05-18_poker_homeohots_True_[8, 16]_[1000.0, 4000.0]_[2, 4]_20_(None, None)/


## Classification with histogram (post-hoc)

In [ ]:
jitter = (None, None)

train_path = f'../Records/output/train/{hots.name}_{num_sample_train}_{jitter}/'
test_path = f'../Records/output/test/{hots.name}_{num_sample_test}_{jitter}/'

testset = HOTS_Dataset(test_path, trainset.sensor_size, transform=transform, dtype=trainset.dtype)
trainset = HOTS_Dataset(train_path, trainset.sensor_size, transform=transform, dtype=trainset.dtype)

score = make_histogram_classification(trainset, testset, N_neuronz[-1])
print(f' Accuracy: {score*100}%')